In [2]:
# So ... each agent also has a type ideally

Thought: with the type, we could set it within the agent itself.
That way only need to pass the type?

I guess that agents don't have access to the model?

SyntaxError: invalid syntax (2988153278.py, line 3)

In [ ]:
# https://stackoverflow.com/questions/19620498/how-to-create-a-utility-class-correctly

In [ ]:
DEBUG = False

def setup_p_txns(total_steps):
    # Mean at 0
    max_std_devs = 5 # 

    if DEBUG:
        # Note: scipy norm has stddev = 1, mean of 0
        # Below shows why we pick max_std_devs = 5
        p_txn = scipy.integrate.quad(
                scipy.stats.norm.pdf, -5, 5)[0] # quad() Returns: value, error
        print('In total we will capture this amount of the gaussian: ', p_txn)
    
    txn_probabilities = []

    std_dev_per_step = 2 * max_std_devs / total_steps

    for step in range(total_steps):
        p_txn = \
            scipy.integrate.quad(
                scipy.stats.norm.pdf, 
                step * std_dev_per_step, (step+1) * std_dev_per_step
            ) # returns: (integration) value, error
        txn_probabilities.append(p_txn[0])
    if DEBUG:
        txn_probabilities[0] = 1 # set p_txn to 1 to always transact
        txn_probabilities[-1] = 1
        txn_probabilities[1:-1] = 0
    return txn_probabilities

# Okay... so... 
We have agents...

Suspicious ones
Not suspicious ones


Transactions should have:


We also want to export the network of transactions

So... the data we want are... 

sender - receiver - amount - timestamp

parameters:

num of agents,
avg txn hr -> so at any given point, the likelihood of making a transaction

avg txn value
distribution of p per agent type 
starting balance




In [ ]:
# export some... network graphs i guess....
# also want to make some classifieirs

In [ ]:
# Model design
import agentpy as ap
import numpy as np

# Visualization
import seaborn as sns
import pandas as pd

import scipy.stats
import datetime
import matplotlib.pyplot as plt

In [23]:
# work
rng = np.random.default_rng()
rng.random()
rng.normal(loc=0, scale=1)

-2.0953431754200875

In [ ]:
class BankAgent(ap.Agent):
    def setup(self, parameters):
        p = parameters
        #self.txn_value_rng = p.rng
        self.txn_probabilities = p.txn_probabilities
        #self.p.type = 'default'

        self.type = self.p.type
        self.acct_balance = self.p.init_acct_balance

        self.txns = pd.DataFrame(
                columns=['timestep', 'sender_id', 'sender_type', 
                 'receiver_id', 'receiver_type', 'amount', 'acct_balance']
        )

    def transact(self, timestep):
        #shift = self.mean * 4 # WARNING hardcoded 4
        p_txn = self.txn_probabilities[timestep-1]
        if np.random.random() < p_txn and self.acct_balance > 0:
            # randomly choose partner
            partner = self.model.agents.random()
            # randomly choose amount
            amount = self.txn_value_rng()

            print('transacted. timestep', timestep, f'ptxn: {p_txn:.3f}',
            'with partner: ', partner.id, 'new balance: ', self.acct_balance)

            # calculations
            partner.acct_balance += amount
            self.acct_balance -= amount

            # record
            self.txns.loc[len(self.txns.index)] = [
                timestep, self.id, self.type, *partner.id, partner.type,
                amount, self.acct_balance]

In [ ]:
        self.txn_value_rng = p.rng
        self.txn_probabilities = p.txn_probabilities
        #self.p.type = 'default'
        self.type = self.p.type
        self.acct_balance = self.p.init_acct_balance

MEAN_TXN_HRS = {'normal':14, 
                'suspicious':22}
MINS_PER_STEP = 15

MEAN_TXN_VALUES = {'normal':250, 
                'suspicious':50}

NUM_AGENTS_PER_TYPE = {'normal': 1000,
    'suspicious': 100
}

parameters = {
    'num_agents_per_type': NUM_AGENTS_PER_TYPE,
    'mean_txn_hrs': MEAN_TXN_HRS,
    'mean_txn_values': MEAN_TXN_VALUES,
    'starting_balance': 100,
    'seed': 42,
    'mins_per_step' : MINS_PER_STEP, # 1 hr
    'steps': 24 * (60/MINS_PER_STEP), # 24 hours * steps per hr
}


# For plotting
def timestep_to_time(timestep):
    date_and_time = datetime.datetime(2022, 10, 31, 0, 0, 0)
    time_elapsed = timestep * (parameters['mins_per_step'])
    time_change = datetime.timedelta(minutes=time_elapsed)
    new_time = date_and_time + time_change
    return new_time

In [ ]:
class BankModel(ap.Model):
    def setup(self):
        self.guassian_p_txns = setup_p_txns() 

        list_agents = []
        for type in ['normal', 'suspicious']:
            mean = self.p.mean_txn_values[type]
            shift_in_steps = mean * (60 / self.p.mins_per_step).astype(int)
            shifted_p_txns = np.concatenate(
                (self.gaussian_p_txns[-shift_in_steps:], 
                self.gaussian_p_txns[:-shift_in_steps]))
            agents = ap.AgentList(
                self, self.p.num_agents_per_type[type], BankAgent)
            agents.
            list_agents.append(agents)



        self.agents = ap.AgentList(
            self, self.normal_agents + self.suspicious_agents)

        probs_1 = 
        def shift_zscores(self):
            shift = self.mean * 4 # WARNING hardcoded 4
            self.shifted_zscores = np.concatenate(
                (self.zscores[-shift:] , self.zscores[:-shift]))


        self.suspicious_agents = ap.AgentList(
            self, self.p.num_agents_per_type['suspicious'], BankAgent)
    

        self.normal_agents.type = 'normal'
        self.suspicious_agents.type = 'suspicious'


        #self.value_rng = np.random.default_rng(self.p.seed)
        #rng.normal(loc=0, scale=1)

    def step(self):
        self.agents.transact(self.t)
        pass

    def update(self):
        #self.record(foo_bar)
        pass

    def end(self):
        self.agents.record('txns')

In [29]:
np.random.default_rng(42).normal()

0.30471707975443135